In [1]:
from flask import Flask, render_template, request, redirect, url_for, flash, session, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_bcrypt import Bcrypt
import hashlib
import hmac
import json
import urllib
import urllib.parse
from urllib.request import urlopen, Request
import requests
from apscheduler.schedulers.background import BackgroundScheduler
import csv
from sqlalchemy import text
import time
from datetime import date, timedelta, datetime

In [7]:
def generate_hourly_datetimes(from_date: str, to_date: str) -> list:

    start_datetime = datetime.strptime(from_date, '%Y-%m-%d %H:%M:%S')
    end_datetime = datetime.strptime(to_date, '%Y-%m-%d %H:%M:%S')
    datetime_list = []
    current_datetime = start_datetime
    while current_datetime <= end_datetime:
        datetime_list.append(current_datetime.strftime('%Y-%m-%d %H:%M:%S'))
        current_datetime += timedelta(hours=1)
    return datetime_list

def datetime_to_timestamp(date_time_str):
    date_time_format = "%Y-%m-%d %H:%M:%S"
    dt_obj = datetime.strptime(date_time_str, date_time_format)
    timestamp = dt_obj.timestamp()
    return int(timestamp)
        
def get_open_weather_historical_info(lat, long, timestamp):
    API = '3bb980e89cfe2a2a202773ebe12144e7'
    url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={long}&dt={timestamp}&appid={API}"
    response = requests.get(url)

    if response.status_code == 200:
        data = json.loads(response.text)
        data = {
        'user': 'fstatazizi@gmail.com',
        'lat': data['lat'],
        'long': data['lon'],
        'timestamp': timestamp,
        'sunrise': data['data'][0]['sunrise'],
        'sunset': data['data'][0]['sunset'],
        'temp': data['data'][0]['temp'],
        'feels_like': data['data'][0]['feels_like'],
        'pressure': data['data'][0]['pressure'],
        'humidity': data['data'][0]['humidity'],
        'dew_point': data['data'][0]['dew_point'],
        'clouds': data['data'][0]['clouds'],
        'visibility': data['data'][0]['visibility'],
        'wind_speed': data['data'][0]['wind_speed'],
        'wind_deg': data['data'][0]['wind_deg'],
        'weather_main': data['data'][0]['weather'][0]['main'],
        'weather_description': data['data'][0]['weather'][0]['description']
    }
    return data

In [2]:
def generate_hourly_datetimes(from_date: str, to_date: str) -> list:

    start_datetime = datetime.strptime(from_date, '%Y-%m-%d %H:%M:%S')
    end_datetime = datetime.strptime(to_date, '%Y-%m-%d %H:%M:%S')
    datetime_list = []
    current_datetime = start_datetime
    while current_datetime <= end_datetime:
        datetime_list.append(current_datetime.strftime('%Y-%m-%d %H:%M:%S'))
        current_datetime += timedelta(hours=1)
    return datetime_list

def datetime_to_timestamp(date_time_str):
    date_time_format = "%Y-%m-%d %H:%M:%S"
    dt_obj = datetime.strptime(date_time_str, date_time_format)
    timestamp = dt_obj.timestamp()
    return int(timestamp)
        
def get_open_weather_historical_info(lat, long, timestamp):
    API = '3bb980e89cfe2a2a202773ebe12144e7'
    url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={long}&dt={timestamp}&appid={API}"
    response = requests.get(url)

    if response.status_code == 200:
        data = json.loads(response.text)
        data = {
        'user': 'fstatazizi@gmail.com',
        'lat': data['lat'],
        'long': data['lon'],
        'timestamp': timestamp,
        'sunrise': data['data'][0]['sunrise'],
        'sunset': data['data'][0]['sunset'],
        'temp': data['data'][0]['temp'],
        'feels_like': data['data'][0]['feels_like'],
        'pressure': data['data'][0]['pressure'],
        'humidity': data['data'][0]['humidity'],
        'dew_point': data['data'][0]['dew_point'],
        'clouds': data['data'][0]['clouds'],
        'visibility': 0,
        'wind_speed': data['data'][0]['wind_speed'],
        'wind_deg': data['data'][0]['wind_deg'],
        'weather_main': data['data'][0]['weather'][0]['main'],
        'weather_description': data['data'][0]['weather'][0]['description']
    }
    return data

In [ ]:
from_date = '2023-06-08 00:00:00'
to_date = '2023-06-09 23:00:00'
date_time_list = generate_hourly_datetimes(from_date, to_date)
lat = '53.276220'
long = '-6.227920'
info = get_open_weather_historical_info(lat, long, '1686186000')
info

In [ ]:
from_date = '2023-06-08 00:00:00'
to_date = '2023-06-09 23:00:00'
date_time_list = generate_hourly_datetimes(from_date, to_date)
lat = '53.276220'
long = '-6.227920'
for date_time in date_time_list:
    timestamp_value = datetime_to_timestamp(date_time)
    info = get_open_weather_historical_info(lat, long, timestamp_value)
    print(f"Timestamp: {timestamp_value}, Date-Time: {date_time}")

In [3]:
def get_historical_data():
    
    row_data = {
    'user': None,
    'date_time': None,
    'timestamp': None,
    'lat': None,
    'long': None,
    'sunrise': None,
    'sunset': None,
    'temp': None,
    'feels_like': None,
    'pressure': None,
    'humidity': None,
    'dew_point': None,
    'clouds': None,
    'visibility': None,
    'wind_speed': None,
    'wind_deg': None,
    'weather_main': None,
    'weather_description': None
}
    from_date = '2024-06-09 23:00:00'
    to_date = '2024-06-14 23:00:00'
    date_time_list = generate_hourly_datetimes(from_date, to_date)
    lat = '53.276220'
    long = '-6.227920'
    for date_time in date_time_list:
        timestamp_value = datetime_to_timestamp(date_time)
        info = get_open_weather_historical_info(lat, long, timestamp_value)
        row_data['user'] = 'fstatazizi@gmail.com'
        row_data['date_time'] = date_time
        row_data['timestamp'] = timestamp_value
        row_data['lat'] = info['lat']
        row_data['long'] = info['long']
        row_data['sunrise'] = info['sunrise']
        row_data['sunset'] = info['sunset']
        row_data['temp'] = info['temp']
        row_data['feels_like'] = info['feels_like']
        row_data['pressure'] = info['pressure']
        row_data['humidity'] = info['humidity']
        row_data['dew_point'] = info['dew_point']
        row_data['clouds'] = info['clouds']
        row_data['visibility'] = info['visibility']
        row_data['wind_speed'] = info['wind_speed']
        row_data['wind_deg'] = info['wind_deg']
        row_data['weather_main'] = info['weather_main']
        row_data['weather_description'] = info['weather_description']
        
        with open('historical_data.csv', mode='a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=row_data.keys())
            writer.writerow(row_data)
            # time.sleep(10)

In [4]:
get_historical_data()

UnboundLocalError: cannot access local variable 'data' where it is not associated with a value